# Задание для DS по разработке PD модели

## Необходиом разработать модель предсказания дефолта. Даны финансовые данные юридических лиц (выручка, активы, пассивы и т.д). 

### Импортируем библиотеки

In [36]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

### Подгружаем данные

In [4]:
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data.csv

/bin/sh: wget: command not found


In [2]:
df = pd.read_csv('PD-data.csv', sep=';')

Размеры выборок

In [3]:
print(df.shape)

(32595, 37)


Количество разных типов фичей

In [4]:
df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [5]:
df.default_12m.value_counts(normalize=True)

0    0.932781
1    0.067219
Name: default_12m, dtype: float64

In [6]:
df.record_id.nunique()

32595

32595 уникальных наблюдений

### Строим бэйзлайн модель

In [24]:
features = ['bus_age','ogrn_age',
                              'head_actual_age', 'adr_actual_age',
                              'cap_actual_age', 'ul_staff_range',
                              'ul_capital_sum', 'ul_founders_cnt',
                              'ul_branch_cnt', 'ul_strategic_flg',
                              'ul_systematizing_flg']

In [25]:
features_df = df.loc[:,features]
features_df['default_12m'] = df['default_12m']

In [26]:
features_df.head()

,bus_age,ogrn_age,head_actual_age,adr_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,198,136,3,3,3,[1-100],50000.0,2,0,0,0,0
1,256,135,12,12,12,[1-100],10000.0,5,0,0,0,0
2,33,33,28,28,28,[1-100],10000.0,2,0,0,0,0
3,185,139,8,8,8,[1-100],300000.0,1,0,0,0,0
4,220,138,27,27,27,[1-100],10000.0,2,0,0,0,0


### Построим карту коллеряций

In [32]:
features_df.corr()

,bus_age,ogrn_age,head_actual_age,adr_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
bus_age,1.000000,0.857296,0.024008,0.024008,0.024008,0.067037,0.209244,0.071920,0.019498,NaN,-0.111336
ogrn_age,0.857296,1.000000,0.112758,0.112758,0.112758,0.031455,0.135316,0.057288,-0.004515,NaN,-0.123441
head_actual_age,0.024008,0.112758,1.000000,1.000000,1.000000,-0.032241,-0.018492,0.066897,-0.011395,NaN,-0.055559
adr_actual_age,0.024008,0.112758,1.000000,1.000000,1.000000,-0.032241,-0.018492,0.066897,-0.011395,NaN,-0.055559
cap_actual_age,0.024008,0.112758,1.000000,1.000000,1.000000,-0.032241,-0.018492,0.066897,-0.011395,NaN,-0.055559
ul_capital_sum,0.067037,0.031455,-0.032241,-0.032241,-0.032241,1.000000,0.029621,0.018203,0.165191,NaN,-0.019556
ul_founders_cnt,0.209244,0.135316,-0.018492,-0.018492,-0.018492,0.029621,1.000000,0.012339,0.035811,NaN,-0.024054
ul_branch_cnt,0.071920,0.057288,0.066897,0.066897,0.066897,0.018203,0.012339,1.000000,-0.000605,NaN,-0.014770
ul_strategic_flg,0.019498,-0.004515,-0.011395,-0.011395,-0.011395,0.165191,0.035811,-0.000605,1.000000,NaN,-0.002575
ul_systematizing_flg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
real_features = ['bus_age', 'ogrn_age', 'head_actual_age',
                 'ul_capital_sum', 'ul_founders_cnt','ul_staff_range']
categ_features = ['ul_staff_range','ul_strategic_flg','ul_branch_cnt']
try_df, scaler = get_dataset(features_df,real_features,categ_features)
X_train, X_test, y_train, y_test = train_test_split(try_df.drop('default_12m',axis=1),
                                                    try_df['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

ValueError: could not convert string to float: '[1-100]'

In [18]:
logit.fit(X_train,y_train)

NameError: name 'logit' is not defined